# Setup

## Packages

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.3 MB/s eta 0:00:00


In [ ]:
import csv
import pandas as pd
from google.colab import drive
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification, BertModel, BertConfig, AdamW

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

from sklearn.model_selection import train_test_split, ParameterGrid

from scipy.spatial.distance import cosine
from scipy.stats import pearsonr

In [ ]:
drive.mount('/content/drive')
FOLDERNAME = 'ColabNotebooks/263/263 Final Project/Data'
%cd drive/My\ Drive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/ColabNotebooks/263/263 Final Project/Data


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

In [ ]:
train = pd.read_csv("train_df.csv")
dev = pd.read_csv("evaluation_df.csv")

# Drop rows with NaN values in 'text1' or 'text2'
train = train.dropna()

In [ ]:
len(train)

4842

In [ ]:
for column in ['Geography','Entities','Time','Narrative','Overall','Style','Tone']:
    train[column] = train[column] - 1

In [ ]:
for column in ['GEO',	'ENT',	'TIME',	'NAR',	'Overall',	'STYLE',	'TONE']:
    dev[column] = dev[column] - 1

In [ ]:
def rescale_inverted(old_value, old_min=0, old_max=3, new_min=1, new_max=-1):
    new_value = ((old_value - old_min) / (old_max - old_min)) * (new_min - new_max) + new_max
    return new_value

In [ ]:
train['Rescaled_Overall'] = train['Overall'].apply(lambda x: rescale_inverted(x, old_min=0, old_max=3, new_min=-1, new_max=1))

dev['Rescaled_Overall'] = dev['Overall'].apply(lambda x: rescale_inverted(x, old_min=0, old_max=3, new_min=-1, new_max=1))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train[['text1','text2']], train[['Geography','Entities','Time','Narrative','Rescaled_Overall','Style','Tone']])

In [ ]:
labels = torch.tensor(y_train.values, dtype=torch.long)

# Bert (Head)

## Train

### Tokenize (Head)

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenize the 'text1' and 'text2' columns
text1_sentences = train['text1'].tolist()
text2_sentences = train['text2'].tolist()

# Tokenize the sentences
text1_inputs = tokenizer(text1_sentences, padding=True, truncation=True, return_tensors="pt")
text2_inputs = tokenizer(text2_sentences, padding=True, truncation=True, return_tensors="pt")


In [ ]:
# Create tensor datasets separately
dataset1 = TensorDataset(text1_inputs['input_ids'], text1_inputs['attention_mask'])
dataset2 = TensorDataset(text2_inputs['input_ids'], text2_inputs['attention_mask'])


In [ ]:
# Set batch size
batch_size = 32

# Create DataLoaders
dataloader1 = DataLoader(dataset1, batch_size=batch_size, shuffle=True)
dataloader2 = DataLoader(dataset2, batch_size=batch_size, shuffle=True)

### Pretrained-Model

In [ ]:
# Initialize the BERT model
model = BertModel.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Initialize an empty list to hold the similarities
similarities = []

for (batch1, batch2) in zip(dataloader1, dataloader2):
    # Unpack the batch and load onto GPU
    text1_inputs = {key: val.to(device) for key, val in zip(['input_ids', 'attention_mask'], batch1)}
    text2_inputs = {key: val.to(device) for key, val in zip(['input_ids', 'attention_mask'], batch2)}

    # Forward pass
    with torch.no_grad():
        text1_outputs = model(**text1_inputs)
        text2_outputs = model(**text2_inputs)

    # Get the embeddings of the [CLS] token which represents the sentence representation
    text1_embeddings = text1_outputs.last_hidden_state[:, 0, :]
    text2_embeddings = text2_outputs.last_hidden_state[:, 0, :]

    # Calculate cosine similarities for each pair of sentences
    for sent1, sent2 in zip(text1_embeddings, text2_embeddings):
        similarity = 1 - cosine(sent1.cpu().numpy(), sent2.cpu().numpy())
        similarities.append(similarity)

KeyboardInterrupt: ignored

In [ ]:
train['similarities'] = similarities

In [ ]:
len(similarities)

4842

### Pearson correlation

In [ ]:
# Calculate Pearson correlation
correlation, _ = pearsonr(train['similarities'], train['Rescaled_Overall'])

# Save the correlation in the DataFrame
train['correlation'] = correlation

In [ ]:
print(np.mean(correlation))

0.01879360899159983


In [ ]:
train.to_csv('train_similarities.csv', index=False)

## Evaluation

### Tokenize (Head)

In [ ]:
# Tokenize the 'text1' and 'text2' columns
text1_sentences_dev = dev['text1'].tolist()
text2_sentences_dev = dev['text2'].tolist()

# Tokenize the sentences
text1_inputs_dev = tokenizer(text1_sentences_dev, padding=True, truncation=True, return_tensors="pt")
text2_inputs_dev = tokenizer(text2_sentences_dev, padding=True, truncation=True, return_tensors="pt")


In [ ]:
# Create tensor datasets separately
dataset1_dev = TensorDataset(text1_inputs_dev['input_ids'], text1_inputs_dev['attention_mask'])
dataset2_dev = TensorDataset(text2_inputs_dev['input_ids'], text2_inputs_dev['attention_mask'])

# Set batch size
batch_size = 32

# Create DataLoaders
dataloader1_dev = DataLoader(dataset1_dev, batch_size=batch_size, shuffle=True)
dataloader2_dev = DataLoader(dataset2_dev, batch_size=batch_size, shuffle=True)

### Pretrained-Model

In [ ]:
# Initialize the BERT model
model = BertModel.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Initialize an empty list to hold the similarities
similarities = []

for (batch1, batch2) in zip(dataloader1_dev, dataloader2_dev):
    # Unpack the batch and load onto GPU
    text1_inputs = {key: val.to(device) for key, val in zip(['input_ids', 'attention_mask'], batch1)}
    text2_inputs = {key: val.to(device) for key, val in zip(['input_ids', 'attention_mask'], batch2)}

    # Forward pass
    with torch.no_grad():
        text1_outputs = model(**text1_inputs)
        text2_outputs = model(**text2_inputs)

    # Get the embeddings of the [CLS] token which represents the sentence representation
    text1_embeddings = text1_outputs.last_hidden_state[:, 0, :]
    text2_embeddings = text2_outputs.last_hidden_state[:, 0, :]

    # Calculate cosine similarities for each pair of sentences
    for sent1, sent2 in zip(text1_embeddings, text2_embeddings):
        similarity = 1 - cosine(sent1.cpu().numpy(), sent2.cpu().numpy())
        similarities.append(similarity)

In [ ]:
len(similarities)

4902

In [ ]:
len(dev)

4902

In [ ]:
dev['similarities'] = similarities

### Pearson correlation

In [ ]:
# Calculate Pearson correlation
correlation, _ = pearsonr(dev['similarities'], dev['Rescaled_Overall'])

# Save the correlation in the DataFrame
dev['Rescaled_correlation'] = correlation

In [ ]:
print(np.mean(correlation))

-0.010094246077528821


In [ ]:
dev.to_csv('dev_similarities.csv', index=False)